#LIB INSTALATION
!pip install yfinance
!pip install crewai
!pip install 'crewai[tools]'
!pip install langchain
!pip install langchain-openai
!pip install langchain-community
!pip install python-dotenv

In [59]:
import json
import os
from datetime import datetime
from crewai import Agent, Task, Crew, Process
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults
from dotenv import load_dotenv

import yfinance as yf

In [60]:
#Yahoo Finance Tool
def fetch_stock_price(ticket):
  stock = yf.download(ticket, start="2023-08-08", end="2024-08-08")
  return stock

yahoo_finance_tool = Tool(
  name = "Yahoo Finance Tool",
  description = "Fetch Stock prices for {ticket} from the last year about a specific stock from API",
  func = lambda ticket: fetch_stock_price(ticket)
)

#DuckDuck Go Tool
search_tool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [61]:
# IMPORT OPENAI LLM - GPT
load_dotenv()
os.environ['OPENAI_API_KEY'] == os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [62]:
stockPriceAnalyst = Agent(
  role = "Senior stock price analyst",
  goal = "Find the {ticket} stock price and analyze trends",
  backstory = "You're highly exprienced in analyze the price of an specific stock and make predictions about the future price and get some external factors and historical logs study if is worth invest in the moment",
  verbose = True,
  llm = llm,
  max_iter = 5,
  memory = True,
  allow_delegation = False,
  tools = [yahoo_finance_tool]
)

newsAnalyst = Agent(
  role = "Stock news analyst",
  goal = "Create a short summary of the market news related to the stock {ticket} company. Specify the current trend - up, down or sideways with the news context. For each requested stock specify a number between 0 and 100 where 0 is bad and 100 is good",
  backstory = "You're a highly experienced in analyzing the market trends and news tracking assets for more then 10 years, you're also master level analytics in the tradictional markets and have deep understanding of human psychology. You understand news, theris titles and information but you look at those with a health dose of skepticism. You consider also the source of the news and check if is fake",
  verbose = True,
  llm = llm,
  max_iter = 10,
  memory = True,
  allow_delegation = False,
  tools = [search_tool]
)

stockAnalystWritter = Agent(
  role = "Senior Stock Analyst Writer",
  goal = "Analyze the trends price and news and write an insightfull compelling and informative long newsletter based on the stock report and price trend.",
  backstory = "You're widely accepted as the best stock analyst in the market. You understand complex concepts and create compelling stories and narratives that resonate with wider audiences. You understand macro factors and combine multiple theories - eg. cycle theory, and fundamental analyses. You're able to hold a multiple opinions when analyzing anything",
  verbose = True,
  llm = llm,
  max_iter = 5,
  memory = True,
  allow_delegation = True,
)

In [63]:
getStockPrice = Task(
  description = "Analyze the stock {ticket} price history and create a trend analyzes of up, down or sideways",
  agent = stockPriceAnalyst,
  expected_output = "Specify the current trend stock price - up , down or sideways eg. stock='APPL, price UP' ",
)

getNews = Task(
  description = f"Take the stock and always include BTC to it or others trend criptos (if not request). Use the search tool to search one individually. The current date is {datetime.now()}. Compose the results into a helpful report",
  agent = newsAnalyst,
  expected_output = "A summary of the overall market and return a sentence for each request asset. Include a good/bad score asset based on the news. Use format: <STOCK ASSET> <SUMMARY NEWS> <TREND PREDICTION> <GOOD/BAD SCORE> ",
)

WriteAnalyses = Task(
  description = "Use the stock price trend and the stock news report to create an analyses and write the newsletter about the {ticket} company. A brief talk and highlight the most important points. focus on the stock price, news and good/bad score. what are the near future considerations? Also, include the previous analyzes of stock trend and news summary.",
  agent = stockAnalystWritter,
  context = [ getStockPrice, getNews ],
  expected_output = "Return a newsletter formatted as markdown in an easy readable manner. It should contain: -3 bullets executive summary, -introduction (set the overall picture and spike up the interest), - main part (provides the meat of the analysis including the news, summary, and good/bad scores), - summary (key facts and concrete future trends prediction about, - up, down or sideways),   ",
)

In [64]:
crew = Crew(
  agents = [ stockPriceAnalyst, newsAnalyst, stockAnalystWritter ],
  tasks = [ getStockPrice, getNews, WriteAnalyses ],
  verbose = True,
  process = Process.hierarchical,
  full_output = True,
  share_crew = False,
  manager_llm = llm,
  max_iter = 15,
)

2024-08-21 00:54:16,568 - 14616 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [65]:
results = crew.kickoff(inputs={'ticket': 'AAPL'})

 [2024-08-21 00:54:16][DEBUG]: == Working Agent: Crew Manager
 [2024-08-21 00:54:16][INFO]: == Starting Task: Analyze the stock AAPL price history and create a trend analyzes of up, down or sideways


> Entering new CrewAgentExecutor chain...


> Finished chain.
 [2024-08-21 00:55:40][DEBUG]: == [Crew Manager] Task output: Agent stopped due to iteration limit or time limit.


 [2024-08-21 00:55:40][DEBUG]: == Working Agent: Crew Manager
 [2024-08-21 00:55:40][INFO]: == Starting Task: Take the stock and always include BTC to it or others trend criptos (if not request). Use the search tool to search one individually. The current date is 2024-08-21 00:54:16.559075. Compose the results into a helpful report


> Entering new CrewAgentExecutor chain...


> Finished chain.
 [2024-08-21 00:57:03][DEBUG]: == [Crew Manager] Task output: Agent stopped due to iteration limit or time limit.


 [2024-08-21 00:57:03][DEBUG]: == Working Agent: Crew Manager
 [2024-08-21 00:57:03][INFO]: == Starting Tas